In [ ]:
# See https://mortendahl.github.io/

import numpy as np
import random

BASE = 10
PRECISION_INTEGRAL = 8
PRECISION_FRACTIONAL = 8
Q = 63207562083765982736 # Ideally a large number
# You cannot represent any number that is greater than Q

PRECISION = PRECISION_INTEGRAL + PRECISION_FRACTIONAL

assert(Q > BASE**PRECISION)

# Fixed Precision Encoding
def encode(rational):
    upscaled = int(rational * BASE**PRECISION_FRACTIONAL)
    field_element = upscaled % Q
    return field_element

# Decoding back to floats
def decode(field_element):
    upscaled = field_element if field_element <= Q/2 else field_element - Q
    rational = upscaled / BASE**PRECISION_FRACTIONAL
    return rational

# secret sharing between three parties
def encrypt(secret):
    first  = random.randrange(Q)
    second = random.randrange(Q)
    third  = (secret - first - second) % Q
    return [first, second, third]

# arg is a list/tuple
def decrypt(sharing):
    return sum(sharing) % Q

def add(a, b):
    c = list()
    for i in range(len(a)):
        c.append((a[i] + b[i]) % Q)
    return tuple(c)

In [ ]:
x = encrypt(encode(0.5)); x

[34378441793392086147, 49578971404501726239, 42457710969688153086]

We get the fixed precision representation of x decomposed into three splits to be shared with three parties.

In [ ]:
y = encrypt(encode(0.75)); y

[929353981137251951, 2160434064255737490, 60117774038447993295]

In [ ]:
z = add(x, y); z

(35307795774529338098, 51739405468757463729, 39367922924370163645)

In [ ]:
decrypt(z)

125000000

In [ ]:
decode(decrypt(z))

1.25

## Encrypted Subtraction and Scalar/Public Multiplication

In [ ]:
field = 4283765287436502436

x = 5
bobs_x_share = 74567658 # Some random number
alices_x_share = field - bobs_x_share + x

In [ ]:
(bobs_x_share + alices_x_share) % field

5

#### What if we have a field size of 10?

In [ ]:
field = 10

x = 5

bobs_x_share = 8
alices_x_share = field - bobs_x_share + x

y = 1

bobs_y_share = 9
alices_y_share = field - bobs_y_share + y

In [ ]:
((bobs_x_share + alices_x_share) + (bobs_y_share + alices_y_share)) % field

6

In [ ]:
((bobs_x_share + alices_x_share) - (bobs_y_share + alices_y_share)) % field

4

When we add two numers inside a field (modulo), whatever the remainder is, also adds up in the process. Subtraction works the same way but in reverse.

### Subtraction

In [ ]:
def sub(a, b):
    c = []
    for i in range(len(a)):
        c.append((a[i]-b[i]) % Q)
    return tuple(c)

In [ ]:
decode(decrypt(sub(encrypt(encode(0.8)), encrypt(encode(0.2)))))

0.6

### Scalar Multiplication

(a+b)*2 = 2a + 2b

In [ ]:
def imul(a, scalar):
    'a is tuple of encrypted shares \
    scalar is a plaintext real number'
    
    c = []
    for i in range(len(a)):
        c.append((a[i]*scalar) % Q)
    return tuple(c)

In [ ]:
x = encrypt(encode(2.5)); x

[15421408681201468554, 61636925121536912936, 49356790365043583982]

In [ ]:
imul(x, 10)

(27798962644482720068, 47501192461475284736, 51114969064073960668)

In [ ]:
decrypt(imul(x, 10))

2500000000

In [ ]:
decode(decrypt(imul(x, 10)))

25.0